In [ ]:
%matplotlib inline

In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import tensorflow as tf

from transformers import pipeline, AutoTokenizer

from scipy.special import softmax

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [ ]:
%store -r DT_rally_speaches_dataset
df = DT_rally_speaches_dataset

In [ ]:
df.head()

In [ ]:
# zero_shot_classifier(
#     df.content[0],
#     candidate_labels=["education", "politics", "business", "america", "election"]
# )
# sentiment_analysis_classifier = classifier = pipeline("sentiment-analysis")
# sentiment_analysis_classifier("I've been waiting for a HuggingFace course my whole life.")
# sentiment_analysis_classifier(df.content[29], truncation=True)

In [6]:
# from transformers import AutoTokenizer
# from transformers import AutoModelForSequenceClassification
# from scipy.special import softmax

In [7]:
#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
checkpoint = 'ProsusAI/finbert'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [8]:
# inputs = tokenizer([df.content[0], df.content[1], 'My name Jeff.!'], padding=True, truncation=True, return_tensors="tf")
# print(inputs['input_ids'][2])
# print(len(inputs['input_ids'][1]))

In [9]:
# from transformers import TFAutoModel

# checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# model = TFAutoModel.from_pretrained(checkpoint)

# outputs = model(inputs)
# print(outputs.last_hidden_state.shape)

# from transformers import TFAutoModelForSequenceClassification

# checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
# outputs = model(inputs)

# print(outputs.logits.shape)

# print(outputs.logits)

In [10]:
# print(len(df.content[0]))

In [11]:
tokens = tokenizer(df.content[1], add_special_tokens=False, return_tensors="tf")
print(tokens)

Token indices sequence length is longer than the specified maximum sequence length for this model (22974 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': <tf.Tensor: shape=(1, 22974), dtype=int32, numpy=array([[2045, 1005, 1055, ..., 4067, 2017, 1012]])>, 'token_type_ids': <tf.Tensor: shape=(1, 22974), dtype=int32, numpy=array([[0, 0, 0, ..., 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(1, 22974), dtype=int32, numpy=array([[1, 1, 1, ..., 1, 1, 1]])>}


In [12]:
tokens['input_ids'].size

22974

In [13]:
tokens['input_ids'].size/510

45.04705882352941

In [14]:
# Setting up id and mask chunks
chunk_size = 510
id_chunks = np.array_split(tokens['input_ids'][0], math.ceil(tokens['input_ids'].size/chunk_size),axis=0)
mask_chunks = np.array_split(tokens['attention_mask'][0], math.ceil(tokens['attention_mask'].size/chunk_size),axis=0)

In [15]:
id_chunks = [np.concatenate(([101],chunk,[102])) for chunk in id_chunks]
mask_chunks = [np.concatenate(([1],chunk,[1])) for chunk in mask_chunks]

# for chunk in id_chunks:
#     print(chunk)

In [16]:
# for chunk in id_chunks:
#     print(len(chunk))

In [17]:
for i in range(len(id_chunks)):
    longest_array = len(max(id_chunks, key=len))
    if len(id_chunks[i]) < longest_array:
        diff = longest_array - len(id_chunks[i])
#         print(diff)
#         print(len(id_chunks[i]))
#         print(longest_array)
        id_chunks[i] = np.concatenate((id_chunks[i],[0]*diff))
        mask_chunks[i] = np.concatenate((mask_chunks[i],[0]*diff))

In [18]:
# id_chunks = [np.concatenate(([101],chunk,[102])) for chunk in id_chunks]

In [19]:
print(len(id_chunks[0]))

502


In [20]:
print(len(id_chunks[10]))

502


In [21]:
print(id_chunks)

[array([  101,  2045,  1005,  1055,  1037,  2843,  1997,  2111,  1012,
        2008,  1005,  1055,  2307,  1012,  4067,  2017,  2200,  2172,
        1012,  4067,  2017,  2200,  2172,  1012,  2008,  1005,  1055,
        1037,  2502,  2177,  1997,  2111,  1012,  2023,  2003,  2006,
        3435,  5060,  1010,  2205,  1012,  4067,  2017,  1012,  2021,
        1045,  1005,  1049, 16082,  2000,  2022,  2182,  2007,  1996,
        3376,  1010,  2307,  1010,  2524, 21398,  2111,  1997,  2023,
        9788,  2110,  1012,  2017,  1005,  2128,  2428,  2524, 21398,
        2137, 11579,  1012,  2008,  1005,  1055,  2054,  2017,  2024,
        1010,  1998,  1037,  2843,  1997,  2111,  4033,  1005,  1056,
        2042,  5845,  2157,  2127,  1045,  2234,  2247,  1012,  2057,
        1005,  2310,  2589,  1037,  2843,  1997,  2147,  1998,  1037,
        2843,  1997,  2204,  2147,  1010,  1998,  2017,  2018,  2115,
        2190,  2095,  2412,  2197,  2095,  1012,  1996,  2110,  2018,
        1996,  2190

In [22]:
print(len(mask_chunks[0]))

502


In [23]:
print(len(mask_chunks[10]))

502


In [24]:
print(mask_chunks)

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [25]:
tf_id_chunks = tf.convert_to_tensor(id_chunks)
tf_mask_chunks = tf.convert_to_tensor(mask_chunks)

# for chunk in tf_id_chunks:
#     print(chunk)

In [26]:
input_dict = {
    'input_ids' : tf_id_chunks,
    'attention_mask' : tf_mask_chunks
}
input_dict

{'input_ids': <tf.Tensor: shape=(46, 502), dtype=int32, numpy=
 array([[  101,  2045,  1005, ...,  3039,  2000,   102],
        [  101,  2031,  2576, ...,  2115,  3707,   102],
        [  101, 10848,  1012, ...,  2061,  2146,   102],
        ...,
        [  101,  2038,  2701, ...,  2031,   102,     0],
        [  101,  2204,  9165, ...,  5135,   102,     0],
        [  101,  1012,  2057, ...,  1012,   102,     0]])>,
 'attention_mask': <tf.Tensor: shape=(46, 502), dtype=int32, numpy=
 array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 0],
        [1, 1, 1, ..., 1, 1, 0],
        [1, 1, 1, ..., 1, 1, 0]])>}

In [27]:
from transformers import TFBertForSequenceClassification #, BertTokenizer

In [28]:
# tokenizer_bert = BertTokenizer.from_pretrained(checkpoint)
model = TFBertForSequenceClassification.from_pretrained(checkpoint)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
outputs = model(input_dict)

In [30]:
print(outputs.logits.shape)

(46, 3)


In [31]:
print(outputs.logits)

tf.Tensor(
[[-0.85311747  0.0836704  -0.35553968]
 [-0.06653055 -0.47242808  0.05270851]
 [-0.34277683 -0.5609694   0.09988356]
 [-0.30890745 -0.5587143   0.06161331]
 [-0.390123   -0.57425654  0.02932104]
 [-0.3612395  -0.7178007   0.09977069]
 [-0.01408508 -0.56430686  0.1762685 ]
 [-0.6167517  -0.40538645 -0.04406214]
 [-0.40341526 -0.6115476   0.09196354]
 [-0.07597276 -0.6629886   0.17202999]
 [-0.58377707 -0.41338414 -0.01301211]
 [-0.46278206 -0.52065974  0.02455419]
 [-0.49393502 -0.6244902   0.05357283]
 [-0.30205807 -0.60320807  0.11705247]
 [-0.00419033 -0.7322003   0.2392601 ]
 [-0.7384055  -0.5434008   0.02720159]
 [-0.15338662 -0.6218648   0.20445377]
 [-0.25412202 -0.672886    0.12175229]
 [-0.38598827 -0.6044763   0.07734649]
 [-0.67897797 -0.50589174  0.03802228]
 [-0.18807192 -0.71136427  0.14162505]
 [-0.30862147 -0.5449529   0.08038348]
 [-0.20766848 -0.6374707   0.16723444]
 [-0.5142963  -0.5079232   0.05083649]
 [-0.63117146 -0.4333797   0.03783348]
 [-0.6750114  

In [32]:
model.config.id2label

{0: 'positive', 1: 'negative', 2: 'neutral'}

In [33]:
predictions = tf.math.softmax(outputs.logits, axis=-1)
print(predictions)

tf.Tensor(
[[0.19243708 0.49105546 0.31650752]
 [0.35803568 0.23858728 0.40337706]
 [0.29754695 0.239219   0.46323404]
 [0.30984306 0.24135266 0.44880423]
 [0.29824567 0.24808812 0.45366618]
 [0.3043441  0.2130651  0.48259082]
 [0.35887322 0.20700592 0.43412077]
 [0.24947655 0.30819422 0.4423292 ]
 [0.28958485 0.23517174 0.47524342]
 [0.3524294  0.1959451  0.4516254 ]
 [0.2528195  0.29978597 0.44739452]
 [0.27997562 0.2642313  0.45579302]
 [0.27727354 0.2433375  0.479389  ]
 [0.30669436 0.22694363 0.466362  ]
 [0.3625143  0.17504688 0.46243882]
 [0.22906275 0.27838376 0.4925535 ]
 [0.3272046  0.20481509 0.46798027]
 [0.32111764 0.21125028 0.46763209]
 [0.2947159  0.23687294 0.46841124]
 [0.2360022  0.2805992  0.48339853]
 [0.33522025 0.19864027 0.46613944]
 [0.3062758  0.24181056 0.4519136 ]
 [0.32201058 0.20951225 0.46847716]
 [0.26552844 0.2672261  0.4672455 ]
 [0.23975052 0.29218602 0.46806344]
 [0.23690526 0.3147502  0.4483445 ]
 [0.27282915 0.23998266 0.48718825]
 [0.30590197 0.25

In [34]:
mean = tf.math.reduce_mean(predictions, axis=0, keepdims=False, name=None)
mean

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.2751    , 0.26839492, 0.45650515], dtype=float32)>

In [159]:
all_tokens = [tokenizer(df.content[i], add_special_tokens=False, return_tensors="tf") for i in range(len(df.content))]
all_tokens[1]

{'input_ids': <tf.Tensor: shape=(1, 22974), dtype=int32, numpy=array([[2045, 1005, 1055, ..., 4067, 2017, 1012]])>, 'token_type_ids': <tf.Tensor: shape=(1, 22974), dtype=int32, numpy=array([[0, 0, 0, ..., 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(1, 22974), dtype=int32, numpy=array([[1, 1, 1, ..., 1, 1, 1]])>}

In [160]:
len(all_tokens)

35

In [161]:
all_tokens[0]['input_ids'][0]

<tf.Tensor: shape=(24289,), dtype=int32, numpy=array([4067, 2017, 1012, ..., 4067, 2017, 1012])>

In [162]:
len(all_tokens)

35

In [163]:
chunk_size = 510
for i in range(len(all_tokens)):
    all_tokens[i]['input_ids'] = np.array_split(all_tokens[i]['input_ids'][0], math.ceil(all_tokens[i]['input_ids'].size/chunk_size),axis=0)
    all_tokens[i]['attention_mask'] = np.array_split(all_tokens[i]['attention_mask'][0], math.ceil(all_tokens[i]['attention_mask'].size/chunk_size),axis=0)
    
#     for j in range(len(all_tokens[i]['input_ids'])):  
#         all_tokens[i]['input_ids'][j] = [np.concatenate(([101],all_tokens[i]['input_ids'][j],[102])) for chunk in all_tokens[i]['input_ids'][j]]
#         all_tokens[i]['attention_mask'][j] = [np.concatenate(([1],all_tokens[i]['attention_mask'][j],[1])) for chunk in all_tokens[i]['attention_mask'][j]]

In [164]:
# for i in range(len(all_tokens)):
#     for j in range(len(all_tokens[i]['input_ids'])):
#         all_tokens[i]['input_ids'][j] = [np.concatenate(([101],all_tokens[i]['input_ids'][j],[102])) for chunk in all_tokens[i]['input_ids'][j]]
#         all_tokens[i]['attention_mask'][j] = [np.concatenate(([1],all_tokens[i]['attention_mask'][j],[1])) for chunk in all_tokens[i]['attention_mask'][j]]

In [165]:
for i in range(len(all_tokens)):
    for j in range(len(all_tokens[i]['input_ids'])):
        all_tokens[i]['input_ids'][j] = np.concatenate(([101],all_tokens[i]['input_ids'][j],[102]))
        all_tokens[i]['attention_mask'][j] = np.concatenate(([1],all_tokens[i]['attention_mask'][j],[1]))

In [ ]:
for i in range(len(id_chunks)):
    longest_array = len(max(id_chunks, key=len))
    if len(id_chunks[i]) < longest_array:
        diff = longest_array - len(id_chunks[i])
#         print(diff)
#         print(len(id_chunks[i]))
#         print(longest_array)
        id_chunks[i] = np.concatenate((id_chunks[i],[0]*diff))
        mask_chunks[i] = np.concatenate((mask_chunks[i],[0]*diff))

In [152]:
for i in range(len(all_tokens)):
    all_tokens[i]['input_ids'] = tf.convert_to_tensor(all_tokens[i]['input_ids'])

ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [168]:
for i in all_tokens[1]['input_ids']:
    print(len(all_tokens[1]['input_ids']))

46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46


In [167]:
all_tokens[0]['input_ids'][0]

array([  101,  4067,  2017,  1012,  4067,  2017,  1012,  4067,  2017,
        2000,  3580,  2343,  7279,  3401,  1012,  2002,  1005,  1055,
        1037,  2204,  3124,  1012,  2057,  1005,  2310,  2589,  1037,
        2307,  3105,  2362,  1012,  1998, 12831,  4234,  1010,  4174,
        1012,  4067,  2017,  1010,  4174,  1012,  2054,  1037,  3377,
        2057,  2018,  1999,  4174,  1012,  2054,  1037,  3377,  2001,
        2008,  1012,  2028,  1997,  1996,  2307,  2015,  1012,  2001,
        2008,  1996,  4602,  3944,  1029,  2021,  1045,  1005,  1049,
       16082,  2000,  2022,  2182,  2007,  5190,  1997,  2524, 21398,
       11579,  2004,  2057,  8439,  1996,  9727,  1997,  4234,  1010,
        1996,  2307,  2791,  1997,  2637,  1998,  1996,  8294,  1997,
        2643,  1012,  4067,  2017,  2200,  2172,  1012,  1998,  2106,
        2017,  5060,  2008,  7955,  2003,  3038, 12831,  4234,  2153,
        1029,  2106,  2017,  5060,  1029,  3038, 12831,  4234,  1012,
        1045,  3342,

In [145]:
len(all_tokens[0]['input_ids'][0])

509

In [146]:
len(all_tokens[34]['input_ids'])

17

In [147]:
print(all_tokens[34]['input_ids'][3])

[  101  1010  2057  2253  2091  1037  2210  2096  2057  2288  2718  2007
  1037  7865  1012  2085  1010 17235  2850  4160  2074  3631  2019  2035
  2051  2501  1012  1055 16275  1010  2074  3631  2019  2035  2051  2501
  1012 14649  2003  1996  7731  1997  1996  7226  2368  3049  1998  2009
  1005  1055  2025  1996  7731  1997  2637  1012  3342  1045  2056  1010
  2057  2097  2196  2031  1037  6102  2406  1012  2006  2281  3822  1012
  1996  2111  1997  5334  1010  2256  2307  2110  1010  2057  2293  5334
  1012  1045  2180  2009  2197  2051  1010  2011  1037  2843  1012  1998
  1045  2963  2057  1005  2128  2183  2000  2663  2009  2011  2130  2062
  2023  2051  1012  2123  1005  1056  5293  1010  2197  2051  1045  5720
  1010  1045  2056  2057  1005  2128  2183  2000  2507  2017  1037  4171
  7659  1010  1045  2435  2017  1012  1045  2056  1010  2057  1005  2128
  2183  2000  2507  2017  1037  5294  7816  7659  1012  2507  2017  2115
  4171  7659  1010  1045  2435  2068  2000  2017  1

In [148]:
len(all_tokens[33]['input_ids'][1])

508

In [ ]:
# chunk_size = 510
# all_id_chunks = np.array_split(all_tokens['input_ids'][0], math.ceil(all_tokens['input_ids'].size/chunk_size),axis=0)
# all_mask_chunks = np.array_split(all_tokens['attention_mask'][0], math.ceil(all_tokens['attention_mask'].size/chunk_size),axis=0)

In [ ]:
# id_chunks = [np.concatenate(([101],chunk,[102])) for chunk in id_chunks]
# mask_chunks = [np.concatenate(([1],chunk,[1])) for chunk in mask_chunks]

In [ ]:
tf_id_chunks = tf.convert_to_tensor(id_chunks)
tf_mask_chunks = tf.convert_to_tensor(mask_chunks)

In [151]:
all_output = model(all_tokens)

ValueError: Exception encountered when calling layer "tf_bert_for_sequence_classification" (type TFBertForSequenceClassification).

Data of type <class 'transformers.tokenization_utils_base.BatchEncoding'> is not allowed only (<class 'tensorflow.python.framework.ops.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>, <class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'>) is accepted for input_ids.

Call arguments received:
  • input_ids=[{'input_ids': ['tf.Tensor(shape=(509,), dtype=int32)', 'tf.Tensor(shape=(508,), dtype=int32)', 'tf.Tensor(shape=(508,), dtype=int32)', 'tf.Tensor(shape=(508,), dtype=int32)', 'tf.Tensor(shape=(508,), dtype=int32)', 'tf.Tensor(shape=(508,), dtype=int32)', 'tf.Tensor(shape=(508,), dtype=int32)', 'tf.Tensor(shape=(508,), dtype=int32)', 'tf.Tensor(shape=(508,), dtype=int32)', 'tf.Tensor(shape=(508,), dtype=int32)', 'tf.Tensor(shape=(